In [12]:
import json
import re
import orgelpredigt_analysis as oa
from rapidfuzz import fuzz
import pandas as pd
import statistics
import io

import datetime
from numpyencoder import NumpyEncoder

In [13]:
date = datetime.datetime.now().strftime("%y-%m-%d_%H:%M")
date

'25-08-22_17:46'

In [14]:
relevant_page_texts = []
#for n in page_nrs:
for n in range(41, 1291):
    with open(f"source_texts/praxis_pietatis_verses/{n}.json") as f:
        page = json.load(f)
    page_info = {}
    page_info[n] = page
    relevant_page_texts.append(page_info)

In [15]:
with open("sermons_with_most_music.json", "r") as f:
    testsermons = json.load(f)

In [16]:
fuzziness = 80

In [17]:
similarity_table = {}
similarity_table['date'] = date
similarity_table['method'] = 'similarity_search'
similarity_table['fuzziness'] = fuzziness


In [19]:
similarity_table['results'] = []
for id in testsermons:
    print(f"Starting with {id}")
    sermon = oa.Sermon(id)

    # perform classification
    hits = []
    all_sents = 0
    for i in range(len(sermon.chunked)):                # for each paragraph
        for j in range(len(sermon.chunked[i])):         # for each sentence
            if " bibel" in sermon.chunked[i][j]["types"]:
                continue
            else:
                all_sents += 1
                query = " ".join(sermon.chunked[i][j]["words"])
                query = re.sub(r'[/.,;:?!]', '', query)
                for page in relevant_page_texts:
                    for pagenr, verses in page.items():
                        for verse in verses:
                            sim_score = fuzz.ratio(query, verse)
                            if sim_score >= fuzziness:
                                hits.append([query, i, j, pagenr, verse, float(f"{sim_score:.2f}")])

    guessed_hits = pd.DataFrame(hits, columns=["Predigt", "Paragraph", "Satz", "Liederbuch", "Liedvers", "Ähnlichkeit"])     # create dataframe
    guessed_hits['Dopplung'] = guessed_hits.groupby('Satz')['Satz'].transform(lambda x: x.duplicated())

    similarity_table['results'].append([id, all_sents, guessed_hits.to_csv()[1:]])

Starting with E000036
Query executed for E100178, but no data found.
Query executed for E081064, but no data found.
Query executed for E100185, but no data found.
Query executed for E081061, but no data found.
Query executed for E100173, but no data found.
Query executed for E081066, but no data found.
Query executed for E100179, but no data found.
Query executed for E081068, but no data found.
Query executed for E080981, but no data found.
Query executed for E080951, but no data found.
Query executed for E080921, but no data found.
Query executed for E100184, but no data found.
Query executed for E100181, but no data found.
Query executed for E081071, but no data found.
Query executed for E100175, but no data found.
Query executed for E081065, but no data found.
Query executed for E081073, but no data found.
Query executed for E081072, but no data found.
Query executed for E100158, but no data found.
Query executed for E100183, but no data found.
Query executed for E100180, but no dat

In [20]:
similarity_table

{'date': '25-08-22_17:46',
 'method': 'similarity_search',
 'fuzziness': 80,
 'results': [['E000036',
   1660,
   'Predigt,Paragraph,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung\n0,halleluja gelobet sey gott,18,0,408,r alleluja gelobt fey gott,88.46,False\n1,der wir vergessen sollen zu keiner stunde,18,3,408,der wir vergessen foln zu keiner stunde,95.0,False\n2,groß ist herr deine güte etc,19,10,640,groß ist herr deine treu,84.62,False\n3,a wenn ich nur dich,21,13,768,wann ich nur dich,88.89,False\n4,suche ich errettung aus der noth,21,26,1145,und mich retten aus der noth,80.0,False\n5,alle die zu ihm treten,21,28,590,die zu ihm tre ten,85.0,False\n6,daß mich nicht treffe sein zorngericht,23,10,1000,daß mich nicht treffe dein gericht,91.67,True\n7,jst voller freud und singen,24,1,903,ift voller freud und singen,92.59,False\n8,sieht lauter sonnenschein die sonne die mir lachet,24,2,903,sieht lauter connenschein die sonne,80.0,False\n9,jst mein herr jesus christ,24,3,82,d mein herr jesu

In [21]:
with open(f'similarity_tables/sim-search_{fuzziness}_{date}.json', "w") as f:
    json.dump(similarity_table, f, ensure_ascii=False)

In [23]:
buffer = io.StringIO(similarity_table['results'][1][2])
df2 = pd.read_csv(buffer)
df2

,Predigt,Paragraph,Satz,Liederbuch,Liedvers,Ähnlichkeit,Dopplung
0,dein geist o gott,11,122,468,ge geist von gott,82.35,False
1,dein geist o gott,11,122,1127,dein wil o gott,81.25,True
2,dein geist o gott,11,122,1133,ah deinen geist o gott,87.18,True
3,bezeugt daß solches frey des ewgen lebens vors...,11,123,864,daß solches frey des ergen les bens vorschmack...,88.68,False
4,zwingt die saiten in cithara,15,51,741,zwingt die sensen in cythara,85.71,False
5,und last die süße musica gantz n b freudenreic...,15,52,741,und lafst die süss se musica gank frendenreich...,86.96,False
6,lobt ihn mit hertz und munde,17,64,900,s lobt ihn mit herz und munde,94.74,False
7,welchs er uns beydes schenckt,17,65,900,welchs er uns bendes schenckt,96.55,False
8,das ist eine sel’ge stunde,17,66,900,das ist ein selge stunde,96.00,False
9,darinn man sein gedenckt,17,67,900,darin man sein gedenekt,93.62,False
